In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("azminetoushikwasi/supplygraph-supply-chain-planning-using-gnns")

print("Path to dataset files:", path)

C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 209k/209k [00:00<00:00, 769kB/s]

Extracting files...
Path to dataset files: C:\Users\alexa\.cache\kagglehub\datasets\azminetoushikwasi\supplygraph-supply-chain-planning-using-gnns\versions\2


In [1]:
import pandas as pd
path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/"

# Nodes
nodes = pd.read_csv(path + "Nodes/Nodes.csv")   
edges = pd.read_csv(path + "Edges/Edges (Plant).csv")

delivery_to_distributor = pd.read_csv(path + "Temporal Data/Unit/Delivery To distributor.csv")
factory_issue = pd.read_csv(path + "Temporal Data/Unit/factory issue.csv")
production = pd.read_csv(path + "Temporal Data/Unit/Production .csv")
sales_order = pd.read_csv(path + "Temporal Data/Unit/Sales order.csv")



In [11]:
%pip install torch_geometric_temporal

  Using cached torch_geometric_temporal-0.56.2-py3-none-any.whl.metadata (1.9 kB)
Using cached torch_geometric_temporal-0.56.2-py3-none-any.whl (102 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Temporal Data/Unit/"

production = pd.read_csv(path + "Production .csv")
factory_issue = pd.read_csv(path + "factory issue.csv")
delivery = pd.read_csv(path + "Delivery To distributor.csv")
sales_order = pd.read_csv(path + "Sales order.csv")

products = [col for col in production.columns if col != "Date"]
product_to_id = {prod: i for i, prod in enumerate(products)}

def transform_temporal(df, mapping):
    df_no_date = df.drop(columns=["Date"])
    return df_no_date.rename(columns=mapping)

X_prod = transform_temporal(production, product_to_id)
X_issue = transform_temporal(factory_issue, product_to_id)
X_delivery = transform_temporal(delivery, product_to_id)
X_sales = transform_temporal(sales_order, product_to_id)  

X_prod_np = X_prod.to_numpy()
X_issue_np = X_issue.to_numpy()
X_delivery_np = X_delivery.to_numpy()
X_sales_np = X_sales.to_numpy()

# shape: [T, N, F] = [time_steps, num_nodes, num_features]
X = np.stack([X_prod_np, X_issue_np, X_delivery_np, X_sales_np], axis=-1)

y = X_sales_np[1:]       
X = X[:-1]

print("X shape:", X.shape)  # (T-1, N, 4)
print("y shape:", y.shape)  # (T-1, N)


X shape: (220, 41, 4)
y shape: (220, 41)


In [19]:
edges = pd.read_csv("./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Edges/Edges (Plant).csv")

edges['node1'] = edges['node1'].astype(str)
edges['node2'] = edges['node2'].astype(str)

all_nodes = pd.concat([edges['node1'], edges['node2']]).unique()
node_to_id = {node: i for i, node in enumerate(all_nodes)}

edges['node1'] = edges['node1'].map(node_to_id)
edges['node2'] = edges['node2'].map(node_to_id)

edge_index = edges[['node1', 'node2']].to_numpy().T.astype(np.int64)

dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=None,
    features=[X[t] for t in range(X.shape[0])],
    targets=[y[t] for t in range(y.shape[0])]
)


In [20]:
from torch_geometric_temporal.signal import temporal_signal_split

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [35]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GCLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GCLSTM(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)  # Unpack (H, C)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [36]:
model = RecurrentGCN(node_features=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [38]:
from tqdm import tqdm

def calculate_mse(model, dataset):
    """Calculate MSE on a dataset"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for time, snapshot in enumerate(dataset):
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            total_loss += torch.mean((y_hat - snapshot.y) ** 2).item()
    model.train()
    return total_loss / (time + 1)

n_epoch = 100  # Number of epochs
for epoch in tqdm(range(n_epoch)):
    model.train()
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)  # Model prediction
        cost += torch.mean((y_hat - snapshot.y) ** 2)                       # MSE loss
    cost = cost / (time + 1)                                                # Average cost
    cost.backward()                                                         # Backpropagation
    optimizer.step()                                                        # Update weights
    optimizer.zero_grad()                                                   # Reset gradients
    
    # Print metrics every 10 epochs
    if (epoch + 1) % 10 == 0:
        train_mse = calculate_mse(model, train_dataset)
        test_mse = calculate_mse(model, test_dataset)
        print(f"\nEpoch {epoch + 1}/{n_epoch}")
        print(f"Train MSE: {train_mse:.4f}")
        print(f"Test MSE: {test_mse:.4f}")

 10%|█         | 10/100 [00:10<01:54,  1.27s/it]


Epoch 10/100
Train MSE: 7356970.7431
Test MSE: 3133420.2960


 20%|██        | 20/100 [00:20<01:34,  1.18s/it]


Epoch 20/100
Train MSE: 7352574.6315
Test MSE: 3130341.1052


 30%|███       | 30/100 [00:32<01:39,  1.42s/it]

...


Epoch 80/100
Train MSE: 7320623.3123
Test MSE: 3107978.8553


 90%|█████████ | 90/100 [01:44<00:10,  1.06s/it]


Epoch 90/100
Train MSE: 7315591.4153
Test MSE: 3104474.4974


100%|██████████| 100/100 [01:53<00:00,  1.13s/it]


Epoch 100/100
Train MSE: 7310729.6606
Test MSE: 3101093.6368
